In [1]:
!pip install numpy pandas matplotlib scikit-learn tensorflow

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import keras.backend as K

%matplotlib inline

2024-05-23 12:17:41.406948: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-23 12:17:41.409931: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-23 12:17:41.449878: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-23 12:17:42.540236: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# Load the CSV file into a DataFrame
data = pd.read_csv('amazon.csv')

# Display the first few rows of the DataFrame
print(data.head())

         Date      Open      High       Low     Close  Adj Close      Volume
0  1997-05-15  0.121875  0.125000  0.096354  0.097917   0.097917  1443120000
1  1997-05-16  0.098438  0.098958  0.085417  0.086458   0.086458   294000000
2  1997-05-19  0.088021  0.088542  0.081250  0.085417   0.085417   122136000
3  1997-05-20  0.086458  0.087500  0.081771  0.081771   0.081771   109344000
4  1997-05-21  0.081771  0.082292  0.068750  0.071354   0.071354   377064000


In [4]:
data['Date'] = pd.to_datetime(data['Date'])
data = data.sort_values('Date')

In [5]:
# Prepare the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

time_step = 100
X, y = [], []

for i in range(time_step, len(scaled_data)):
    X.append(scaled_data[i-time_step:i, 0])
    y.append(scaled_data[i, 0])

X, y = np.array(X), np.array(y)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Split the data into training and testing sets
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size
X_train, X_test = X[0:train_size], X[train_size:len(X)]
y_train, y_test = y[0:train_size], y[train_size:len(y)]


In [6]:
# Define the model
def create_model():
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

model = create_model()


/usr/local/python/3.10.13/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 10s 52ms/step - loss: 6.5775e-04 - val_loss: 0.0018
Epoch 2/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 10s 50ms/step - loss: 9.6551e-05 - val_loss: 9.8994e-04
Epoch 3/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 8s 52ms/step - loss: 7.6120e-05 - val_loss: 0.0017
Epoch 4/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 10s 52ms/step - loss: 7.5448e-05 - val_loss: 0.0016
Epoch 5/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 8s 51ms/step - loss: 5.5708e-05 - val_loss: 9.1060e-04
Epoch 6/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 10s 52ms/step - loss: 5.7165e-05 - val_loss: 8.1550e-04
Epoch 7/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step - loss: 5.2575e-05 - val_loss: 0.0071
Epoch 8/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step - loss: 7.2273e-05 - val_loss: 0.0020
Epoch 9/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step - loss: 7.4712e-05 - val_loss: 9.1167e-04
Epoch 10/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 10s 50ms/step - loss: 5.8539e-05 - val_loss: 0.0034
Epoch 11/20
161/161 ━━━━━━━━━━━━━━━━━━━━ 10s 50ms/step - loss: 5.339

In [10]:
# Monte Carlo Dropout
def mc_dropout_prediction(model, X, n_iter=100):
    f = K.function([model.input, K.learning_phase()], [model.output])
    result = [f([X, 1])[0] for _ in range(n_iter)]
    return np.array(result)

# Make predictions with Monte Carlo Dropout
mc_samples_train = mc_dropout_prediction(model, X_train)
mc_samples_test = mc_dropout_prediction(model, X_test)


AttributeError: module 'keras.backend' has no attribute 'function'

In [9]:
# Calculate mean and uncertainty
train_predict_mean = mc_samples_train.mean(axis=0)
train_predict_std = mc_samples_train.std(axis=0)
test_predict_mean = mc_samples_test.mean(axis=0)
test_predict_std = mc_samples_test.std(axis=0)

# Inverse transform to get actual values
train_predict_mean = scaler.inverse_transform(train_predict_mean)
test_predict_mean = scaler.inverse_transform(test_predict_mean)
train_actual = scaler.inverse_transform(y_train.reshape(-1, 1))
test_actual = scaler.inverse_transform(y_test.reshape(-1, 1))


NameError: name 'mc_samples_train' is not defined

In [ ]:
# Plotting
plt.figure(figsize=(14, 5))
plt.plot(data['Date'], scaler.inverse_transform(scaled_data), label='Actual')
plt.plot(data['Date'][time_step:train_size+time_step], train_predict_mean, label='Train Predict Mean')
plt.plot(data['Date'][train_size+time_step:], test_predict_mean, label='Test Predict Mean')
plt.fill_between(data['Date'][train_size+time_step:], 
                 test_predict_mean - 2 * test_predict_std, 
                 test_predict_mean + 2 * test_predict_std, 
                 color='red', alpha=0.2, label='Test Prediction Uncertainty')
plt.title('Stock Price Prediction with Monte Carlo Dropout')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.show()
